In [ ]:
# 🌐 Telco Network Performance Analysis

This notebook demonstrates how to analyze network performance data using Snowflake and AI capabilities for telecommunications operations.

## Objectives:
- Analyze network latency and throughput across regions
- Identify performance bottlenecks and optimization opportunities
- Use Cortex AI for predictive analytics on network performance
- Generate insights for network planning and investment decisions


In [ ]:
# Import required libraries
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("📊 Libraries imported successfully!")
print(f"🔗 Snowpark version: {snowpark.__version__}")


In [ ]:
# Get Snowpark session
session = snowpark.context.get_active_session()
print(f"❄️ Connected to Snowflake: {session.get_current_account()}")
print(f"🗄️ Current database: {session.get_current_database()}")
print(f"📂 Current schema: {session.get_current_schema()}")


In [ ]:
## 📈 Network Performance Overview


In [ ]:
# Get network performance summary
print("📊 Analyzing network performance metrics...")

# Get current database and schema for fully qualified table names
current_db = session.get_current_database()
current_schema = session.get_current_schema()

network_summary_query = f"""
SELECT 
    network_type,
    region,
    COUNT(DISTINCT cell_tower_id) as tower_count,
    AVG(latency_ms) as avg_latency_ms,
    AVG(throughput_mbps) as avg_throughput_mbps,
    AVG(packet_loss_percent) as avg_packet_loss,
    AVG(uptime_percent) as avg_uptime
FROM {current_db}.{current_schema}.NETWORK_PERFORMANCE 
GROUP BY network_type, region
ORDER BY network_type, avg_latency_ms
"""

try:
    network_summary_df = session.sql(network_summary_query).to_pandas()
    print("✅ Network Performance Summary:")
    display(network_summary_df)
    
    if network_summary_df.empty:
        print("📭 No network performance data found. Please check if data has been loaded.")
        print("💡 Run the configure_attendee_account.template.sql script to load sample data.")
        
except Exception as e:
    print(f"❌ Error executing query: {e}")
    print(f"🔍 Trying to query: {current_db}.{current_schema}.NETWORK_PERFORMANCE")
    print("💡 Solutions:")
    print("1. Run configure_attendee_account.template.sql to create tables and load data")
    print("2. Verify you have SELECT permissions on the NETWORK_PERFORMANCE table")
    print("3. Check that you're in the correct database and schema context")


In [ ]:
# Create network performance visualization
print("📈 Creating network performance dashboard...")

try:
    if 'network_summary_df' in locals() and not network_summary_df.empty:
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Average Latency by Network Type', 'Throughput by Region', 
                           'Network Uptime Comparison', 'Packet Loss Analysis')
        )

        # Latency by network type
        latency_by_type = network_summary_df.groupby('NETWORK_TYPE')['AVG_LATENCY_MS'].mean().reset_index()
        fig.add_trace(
            go.Bar(x=latency_by_type['NETWORK_TYPE'], y=latency_by_type['AVG_LATENCY_MS'], 
                   name='Latency (ms)', marker_color='lightblue'),
            row=1, col=1
        )

        # Throughput by region
        throughput_by_region = network_summary_df.groupby('REGION')['AVG_THROUGHPUT_MBPS'].mean().reset_index()
        fig.add_trace(
            go.Bar(x=throughput_by_region['REGION'], y=throughput_by_region['AVG_THROUGHPUT_MBPS'],
                   name='Throughput (Mbps)', marker_color='lightgreen'),
            row=1, col=2
        )

        fig.update_layout(height=600, title_text="📊 Network Performance Metrics Dashboard", showlegend=False)
        fig.show()
        
        print("\n💡 Network Performance Insights:")
        print("🔵 Lower latency = Better user experience")
        print("📈 Higher throughput = Better data capacity")
        print("🎯 Use these metrics for network optimization and planning")
    else:
        print("⚠️ No network performance data available for visualization")
        print("🔧 Please ensure the NETWORK_PERFORMANCE table exists and contains data")
        print("📋 Sample data structure expected:")
        print("   - NETWORK_TYPE (text: 5G, 4G, 3G)")
        print("   - REGION (text)")
        print("   - CELL_TOWER_ID (integer)")
        print("   - LATENCY_MS (decimal)")
        print("   - THROUGHPUT_MBPS (decimal)")
        print("   - PACKET_LOSS_PERCENT (decimal)")
        print("   - UPTIME_PERCENT (decimal)")
        
except Exception as e:
    print(f"❌ Error creating visualization: {e}")
    print("💡 Make sure the previous cell executed successfully")
